# Home assignment 1

You should work on the assignement in groups/teams of 3 participants. 

Upload your solution as a jupyter notebook to moodle by Wednesday, 20th of November 23:59h. (The deadline is strict)
It is sufficient if one student of each team submits the solution.

Do not forget to specify the names of all contributing students in the jupyter notebook.

You should add comments to your code where necessary and print the relevant results. You should also always test your code on self-chosen examples.

# WordNet path similarity

In [83]:
from nltk.corpus import wordnet as wn
from collections import deque

def find_all_hypernyms(synset):
    # construct a quese,which stores a list of key-value(hypernym-depth)
    queue = deque([(synset, 0)]) 
    path = {} 
    while queue: # find all hypernyms using breadth-first search
        s, depth = queue.popleft()
        if s in path: 
        # if the hypernym already exists in the path{}, then ingonre it (beacause we wan to find the samllest Depth)
            continue
        # s(hypernyms) is the key in path{}, depth is the value in path{}
        path[s] = depth
        depth = depth + 1
        queue.extend((hyper, depth) for hyper in s._hypernyms())
    return path

def compute_simpath(path_synset_1, path_synset_2):
    for key1 in path_synset_1:
        for key2 in path_synset_2:
            # compare two dictionaries, find the lowest common hypernyms of these two synset
            if key1 == key2 :  # shortest path
                # compute the minmal path_length between these two synset
                path_length = path_synset_1[key1] + path_synset_2[key2]
                return 1 / (path_length + 1) # compute sim_path
    return 0  # if don't have common hypernyms, path_length between two synsets is infinity, sim_path is 0

def pathSimilarity(word_1,word_2):
    simpath_list = [] # stores sim_path of any pair of synsets
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    for synset1 in synsets_1:
        for synset2 in synsets_2:
            path_1 = find_all_hypernyms(synset1)
            path_2 = find_all_hypernyms(synset2)
            #print(compute_simpath(hypernyms_1,hypernyms_2))
            simpath_list.append(compute_simpath(path_1, path_2))
    # print(simpath_list)
    # find the maximal sim_path, which represents the similarity of two words
    word_simlarity = max (simpath_list)
    return word_simlarity

#test cases
print(pathSimilarity('dog','dog'))
print(pathSimilarity('dog','cat'))
print(pathSimilarity('fund','budget'))
print(pathSimilarity('bank','beach'))
print(pathSimilarity('people','dog'))
# synset_1.path_similarity(synset_2)

1.0
0.2
0.5
0.25
0.1


# Markov chain model

In [17]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

def textNormalization(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize (text)
    for i,word in enumerate (words):
        #consider number or number-word combination as a real word
        if re.match('[a-z]+|[A-Z]+|[0-9]+',word):# use regex to judge if the word is a puntuation
            # lemmatize the word. eg: words=>word 
            words[i] = lemmatizer.lemmatize(word)
        else: # if word is a puntuation,remove it from list
            words.remove(word)
    return words

In [19]:
#test case
text = "Tokenize this sentence into sentences and words. This is the task Mr. Lemmerich gave to you. Why, i.e., what the purpose of this, is remains unknown."
print(textNormalization(text))

['Tokenize', 'this', 'sentence', 'into', 'sentence', 'and', 'word', 'This', 'is', 'the', 'task', 'Mr.', 'Lemmerich', 'gave', 'to', 'you', 'Why', 'i.e.', 'what', 'the', 'purpose', 'of', 'this', 'is', 'remains', 'unknown']


In [22]:
from collections import defaultdict
import nltk

def first_order_markov(training_text, laplace_correction = 1): #laplace add-1 estimation
    prob_dict = defaultdict( lambda: 0.001 ) # set inital value 0.001
    words_of_text = textNormalization(training_text)# compute frequency of each word
    V = len(set(words_of_text)) # Vocabulary size
    words_freq = nltk.FreqDist(words_of_text)# dictionary: key = word, value = frequency
    bi_grams = list(nltk.bigrams(words_of_text))# compute frequency of each bigrams
    bi_grams_freq = nltk.FreqDist(bi_grams) # ditionary : key = bigrams(), value = frequency
    # comute the condition probaality of last word and next word
    for word in words_freq.keys():
        for (last_word, next_word) in bi_grams_freq.keys():
            if word == last_word :
                # add-1 estimation
                prob = (bi_grams_freq[(last_word, next_word)] + 1) / (words_freq[word] + V) 
                if (last_word, next_word) not in prob_dict.keys():
                    prob_dict[(last_word, next_word)] = prob                   
# this part of code is to compute the conditional probality of word follwed by itself,which doesn't exist in Bigarms
#             elif word != last_word and word != next_word:
#                 prob = 1/(words_freq[word] + V)
#                 if (word, word) not in prob_dict.keys():
#                     prob_dict[(word,word)] = prob
    return prob_dict

def second_order_markov(training_text, laplace_correction = 1): # laplace add-1 estimation
    words_of_text = textNormalization(training_text)
    prob_dict = defaultdict( lambda: 0.001 ) # set inital value 0.001
    bi_grams = list(nltk.bigrams(words_of_text))
    bi_grams_freq = nltk.FreqDist(bi_grams) # ditionary : key = bigrams(), value = frequency
    V = len(set(bi_grams)) # vocablary size of bigrams
    tri_grams = list(nltk.ngrams(words_of_text, 3))
    tri_grams_freq = nltk.FreqDist(tri_grams)# ditionary : key = trigrams(), value = frequency
    # compute conditional probability
    for (last_word_1, next_word) in bi_grams_freq.keys():
        for (last_1, last_2, next_1) in tri_grams_freq.keys():
            if last_word_1 == last_1 and next_word == last_2:
                prob = (tri_grams_freq[(last_1, last_2, next_1)] + 1)/(bi_grams_freq[(last_word_1, next_word)] + V ) 
                if (last_1, last_2, next_1) not in prob_dict:
                    prob_dict[last_1, last_2, next_1] = prob
    return prob_dict

# perplexity of  first_order_markov model
model = first_order_markov(text)
def perplexity (evaluation_text, model): 
    test_set = textNormalization(evaluation_text)
    bi_grams_test = list(nltk.bigrams(test_set))
    perplexity = 1
    N = 0
    for (last_word, next_word) in bi_grams_test:
        N = N + 1
        perplexity = perplexity * (1 / model[(last_word, next_word)])
    perplexity = pow (perplexity, 1 / float(N))
    return perplexity

# test cases
text  = "The web can be thought of as a huge corpus of unannotated text.The web can be thought of as a huge corpus of unannotated text. Web search engines provide an efficient means of searching this large quantity of text for relevant linguistic examples. The main advantage of search engines is size: since you are searching such a large set of documents, you are more likely to find any linguistic pattern you are interested in."
print(first_order_markov(text), '\n')     
print(second_order_markov(text), '\n')
evaluation_text_1 = 'The web can be thought of as a huge corpus of unannotated text.'
evaluation_text_2 = 'This is a Web.'
evaluation_text_3 = 'Web search engines provide an apple.'
print(perplexity(evaluation_text_1, model))
print(perplexity(evaluation_text_2, model))
print(perplexity(evaluation_text_3, model))

defaultdict(<function first_order_markov.<locals>.<lambda> at 0x00000205B48CBAF8>, {('The', 'web'): 0.0425531914893617, ('The', 'main'): 0.0425531914893617, ('web', 'can'): 0.06382978723404255, ('can', 'be'): 0.06382978723404255, ('be', 'thought'): 0.06382978723404255, ('thought', 'of'): 0.06382978723404255, ('of', 'a'): 0.05660377358490566, ('of', 'unannotated'): 0.05660377358490566, ('of', 'searching'): 0.03773584905660377, ('of', 'text'): 0.03773584905660377, ('of', 'search'): 0.03773584905660377, ('of', 'document'): 0.03773584905660377, ('a', 'a'): 0.06, ('a', 'huge'): 0.06, ('a', 'large'): 0.04, ('huge', 'corpus'): 0.06382978723404255, ('corpus', 'of'): 0.06382978723404255, ('unannotated', 'text.The'): 0.0425531914893617, ('unannotated', 'text'): 0.0425531914893617, ('text.The', 'web'): 0.043478260869565216, ('text', 'Web'): 0.0425531914893617, ('text', 'for'): 0.0425531914893617, ('Web', 'search'): 0.043478260869565216, ('search', 'engine'): 0.06382978723404255, ('engine', 'provi

# Viterbi Algorithm

In [ ]:
from nltk.tokenize import word_tokenize
from collections import defaultdict
import numpy as np

def Viterbi(State_trans_prob, Word_emission_prob, sentence):
    words = word_tokenize(sentence)
    word_tag = {} # {'word':'tag'}
    word_tag_prob = {{}} #{'word':{'tag':prob}}
    
    # initialization, find the tag of first element in words
    first_tag = {}
    for (word, tag), val in Word_emission_prob.items():
        if words == word[0]:
            prob = val * State_trans_prob[('<s>',tag)]
            first_tag[tag] = prob
    first_prob = max(first_tag.values())
    for k,v in first_tag.items():
        if v == max_prob:
            word_tag[words[0]] = k
            word_tag_prob[word[0]][k] = v
    
    # select nodes of each level, constuct path 
    probability = first_prob # init probailty before first iteration
    word_tags = defaultdict(list)
    for i in range(len(words)-1):
      for (w, tag),v in Word_emission_prob.items():
            if w == word[i+1]:
                word_tags[w].append(tag)
         
#             for (tag_1,tag_2) in State_trans_prob.items(): 
#                 if word[i+1] == w and 
#                     proba = probability * v * 
                
        
    
    
    

In [ ]:
State_trans_prob = {('<s>','NNP'):0.2767,('<s>','MD'):0.006,('<s>','VB'):0.0031,('<s>','JJ'):0.0453,('<s>','NN'):0.0449,
                   ('<s>','RB'):0.0510,('<s>','DT'):0.2026,
                   ('NNP','NNP'):0.3777,('NNP','MD'):0.0110,('NNP','VB'):0.0009,('NNP','JJ'):0.0084,('NNP','NN'):0.0584,
                   ('NNP','RB'):0.0090,('NNP','DT'):0.0025,
                   ('MD','NNP'):0.0008,('MD','MD'):0.0002,('MD','VB'):0.7968,('MD','JJ'):0.0005,('MD','NN'):0.0008,
                   ('MD','RB'):0.1698,('MD','DT'):0.0041,
                   ('VB','NNP'):0.0322,('VB','MD'):0.0005,('VB','VB'):0.0050,('VB','JJ'):0.0837,('VB','NN'):0.0615,
                   ('VB','RB'):0.0514,('VB','DT'):0.2231,
                   ('JJ','NNP'):0.0366,('JJ','MD'):0.0004,('JJ','VB'):0.0001,('JJ','JJ'):0.0733,('JJ','NN'):0.4509,
                   ('JJ','RB'):0.0036,('JJ','DT'):0.0036,
                   ('NN','NNP'):0.0096,('NN','MD'):0.0176,('NN','VB'):0.0014,('NN','JJ'):0.0086,('NN','NN'):0.1216,
                   ('NN','RB'):0.0177,('NN','DT'):0.0068,
                   ('RB','NNP'):0.0068,('RB','MD'):0.0102,('RB','VB'):0.1011,('RB','JJ'):0.1012,('RB','NN'):0.0120,
                   ('RB','RB'):0.0728,('RB','DT'):0.0479,
                   ('DT','NNP'):0.1147,('DT','MD'):0.0021,('DT','VB'):0.0002,('DT','JJ'):0.2157,('DT','NN'):0.4744,
                   ('DT','RB'):0.0102,('DT','DT'):0.0017}

In [ ]:
Word_emission_prob = {('Janet','NNP'):0.000032, ('will','MD'):0.308431,('will','VB'):0.000028,('will','NN'):0.0002,
                     ('back','VB'):0.000672,('back','JJ'):0.00034,('back','NN'):0.000223,('back','RB'):0.010446,
                     ('the','NNP'):0.000048,('the','DT'):0.506099,('bill','VB'):0.000028,('bill','NN'):0.002337}
# assume the missing entries are 0

In [ ]:
Sentence = 'Janet will back the bill'
Viterbi(State_trans_prob, Word_emission_prob, Sentence)

# POS tagging with nltk

In [16]:
import nltk
def posTag(sentence):
    words = nltk.word_tokenize(sentence)
    result = nltk.pos_tag(words)
    return result

In [17]:
Sentence = 'Janet will back the bill'
posTag(Sentence)

[('Janet', 'NNP'),
 ('will', 'MD'),
 ('back', 'VB'),
 ('the', 'DT'),
 ('bill', 'NN')]